In [ ]:
from tqdm import tqdm
import pandas as pd
import plotly.express as px
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np
import pytz
import math

In [ ]:
start_date = datetime(2024, 8, 13,0, 0, 0).replace(tzinfo=pytz.UTC)
end_date = datetime.today().replace(tzinfo=pytz.UTC)

host=''
port=8086
username = ''
password = ''
dbname = ''

est = pytz.timezone('US/Eastern')

entity_id = 'thermostat_extension'

In [ ]:
from influxdb import DataFrameClient
from datetime import datetime
import pandas as pd

class InfluxStateReader:
    @staticmethod
    def _get_query(s, e, entity):
        return f"SELECT * FROM \"homeassistant\".\"autogen\".\"state\" WHERE time >= \'{s}\'  AND (\"entity_id\"='{entity}')"
    @staticmethod
    def _get_query_all(s, e):
        return f"SELECT * FROM \"homeassistant\".\"autogen\".\"state\" WHERE time >= \'{s}\'"
    
    def __init__(self, host, port, username, password, dbname):
        self._client = DataFrameClient(host=host, port=port, username=username, password=password)
        self._client.switch_database(dbname)
    
    def query_data(self, start_date:datetime, end_date:datetime, entity = None):
        s = start_date.strftime("%Y-%m-%d")
        e = end_date.strftime("%Y-%m-%d")
        if entity is None:
            return self._client.query(InfluxStateReader._get_query_all(s, e))            
        return self._client.query(InfluxStateReader._get_query(s, e, entity))

In [ ]:
NB_DAY_BATCH = 10
temp_reader = InfluxStateReader(host, port, username, password, dbname)
nb_months = (end_date - start_date).days / NB_DAY_BATCH
import_ranges = [(start_date + relativedelta(days=NB_DAY_BATCH * offset), start_date + relativedelta(days=NB_DAY_BATCH * (offset+1))) for offset in range(math.ceil(nb_months))]
stack = [temp_reader.query_data(s, e, entity_id) for (s,e) in tqdm(import_ranges)]

In [ ]:
df_stack = pd.concat([pd.DataFrame(s['state']) for s in stack])#['friendly_name_str']
df_stack.index = df_stack.index.tz_convert(est)

In [ ]:
df_stack.head()

In [ ]:
df_stack.hvac_action_str

In [ ]:
df = df_stack[['entity_id', 'temperature', 'current_temperature']].reset_index(names=['Time']).set_index(['entity_id', 'Time']).sort_index()

In [ ]:
df_b = df.loc[entity_id].reset_index().drop_duplicates().set_index('Time').resample('5Min').ffill()
df_b = df_b.sort_index()
df_b = df_b.ffill()
df_b['Date'] = [i.date() for i in df_b.index]
df_b['Time'] = [i.time() for i in df_b.index]
df_b = df_b.reset_index(drop=True)
df_b = pd.pivot_table(df_b, index='Time', values=['temperature', 'current_temperature'], aggfunc=[np.mean, lambda x : np.percentile(x, 25), lambda x : np.percentile(x, 75)])
df_b.columns = ['mean current_temperature', 'mean temperature', ' lower current_temperature', 'lower temperature', 'upper current_temperature', 'upper temperature']

In [ ]:
df_b

In [ ]:
df_b.columns

In [ ]:
fig  = px.line(df_b, title='Average Use')
fig['data'][0]['line']['color']='red'
fig['data'][1]['line']['color']='blue'
fig['data'][2]['line']['color']='red'
fig['data'][3]['line']['color']='blue'
fig['data'][4]['line']['color']='red'
fig['data'][5]['line']['color']='blue'
fig.show()

In [ ]:
df.loc[entity_id].reset_index().drop_duplicates().set_index('Time').loc['2024-11-20':].plot()